# Invoice Automation and Payment Reporting System

This notebook automates the process of generating invoices and payment summaries for a freelance writing platform.  
It combines multiple Excel sheets (each representing a writing account such as Titania, Flora, Alyssa, etc.), cleans and processes the data, identifies duplicates, and generates:
- Mother Sheet (all orders)
- Bidder Sheets
- Writer Sheets
- Summary Sheet
- Duplicate Report


## Step 1: Import Required Libraries

In this step, we import the Python libraries that we’ll use throughout the project.  
- **pandas**: for data analysis and working with Excel files  
- **numpy**: for numerical calculations  



In [3]:
import pandas as pd
import numpy as np

## Step 2: Load and Combine Account Sheets

In this step, we load data from multiple Excel sheets — each representing a different writing account (Flora, Titania, Alyssa, Beryl, and Magdalene).  
We use a loop to:
1. Read each sheet individually  
2. Add a new column named **Account** to remember which sheet each record came from  
3. Combine all sheets into one master dataset


In [4]:
# Path to the Excel file
filepath = r"C:\Users\HP\Downloads\Order Edusson (2).xlsx"

# List of account sheet names
accounts = ['Flora', 'Titania', 'Alyssa', 'Beryl', 'Magdalene']

# Create an empty list to hold data temporarily
combined = []

# Loop through each sheet
for name in accounts:
    df = pd.read_excel(filepath, sheet_name=name)
    df['Account'] = name
    df = df.drop(columns=['Unnamed: 19'], errors='ignore')  # drop unwanted column if exists
    combined.append(df)

# Combine all into one DataFrame
df = pd.concat(combined, ignore_index=True)

# View first few rows
df.head()


,Date Bid,Assigned?,Order Number,Order Name / Title,Number of Pages,Bid Amount ($),Due Date (Client),Bidder,Assigned Writer,Writer Deadline,Uploaded?,Editor,Fine ($),Fine To,Writer Pay (KES),Bidder Pay (KES),My Share (KES),Notes / Remarks,Paid Status,Account
0,2025-11-21,False,7576309.0,Assessment Deliverable,2.0,8.0,7day,Kevin,Dennis,2025-11-21 00:00:00,True,Select Editor,NaN,Select,400.0,130.0,470.0,NaN,0.0,Flora
1,2025-11-22,False,7576717.0,the scarlet letter,2.0,8.0,2 days,Kurash,Dennis,2025-11-21 00:00:00,True,Select Editor,NaN,Select,400.0,130.0,470.0,NaN,0.0,Flora
2,2025-11-23,False,7578230.0,Sociology,3.0,12.0,2025-11-25 00:00:00,Maronga,Dennis,NaT,True,Select Editor,NaN,Select,600.0,195.0,705.0,NaN,0.0,Flora
3,2025-12-01,False,7586754.0,Reflection,3.0,15.0,1 day,Stano,Dennis,NaT,False,Select Editor,NaN,NaN,600.0,195.0,1080.0,NaN,0.0,Flora
4,2025-12-12,False,7599086.0,Other,2.0,8.0,7 days,Stano,Dennis,NaT,False,Select Editor,NaN,NaN,400.0,130.0,470.0,NaN,0.0,Flora


## Step 3: Data Cleaning and Formatting

Here, we clean and standardize the dataset:
- Rename long column names to simpler, consistent names  
- Convert date columns into proper datetime format  
- Ensure numeric columns are in the correct numeric data type


In [5]:
# Rename columns for consistency
df = df.rename(columns={
    'Bid Amount ($)': 'Bid_Amount_USD',
    'Writer Pay (KES)': 'Writer_Pay_KES',
    'Bidder Pay (KES)': 'Bidder_Pay_KES',
    'My Share (KES)': 'My_Share_KES',
    'Paid Status': 'Paid_Status',
    'Fine ($)': 'Fine_USD'
})

# Convert date columns to datetime
for col in ['Date Bid', 'Writer Deadline']:
    df[col] = pd.to_datetime(df[col], errors='coerce')

# Convert numeric columns to float
for col in ['Bid_Amount_USD', 'Writer_Pay_KES', 'Bidder_Pay_KES', 'My_Share_KES', 'Fine_USD']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Check data types and completeness
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4996 entries, 0 to 4995
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Date Bid            61 non-null     datetime64[ns]
 1   Assigned?           4996 non-null   bool          
 2   Order Number        62 non-null     float64       
 3   Order Name / Title  59 non-null     object        
 4   Number of Pages     62 non-null     float64       
 5   Bid_Amount_USD      62 non-null     float64       
 6   Due Date (Client)   62 non-null     object        
 7   Bidder              98 non-null     object        
 8   Assigned Writer     99 non-null     object        
 9   Writer Deadline     11 non-null     datetime64[ns]
 10  Uploaded?           4996 non-null   bool          
 11  Editor              103 non-null    object        
 12  Fine_USD            1 non-null      float64       
 13  Fine To             16 non-null     object      

## Step 5: Filter Data by Date Range

In this step, we create a reusable function that filters the dataset between any two dates.  
This makes it easy to generate reports for specific weeks, months, or payment periods.


In [10]:
# Define a function to filter data between two dates
def filter_by_date(df, start_date, end_date):
    start = pd.to_datetime(start_date)
    end = pd.to_datetime(end_date)
    mask = (df['Date Bid'] >= start) & (df['Date Bid'] <= end)
    return df.loc[mask]

# Example usage
filtered_df = filter_by_date(df, '2025-11-20', '2025-11-25')

# Check the size of the filtered dataset
filtered_df.shape


(18, 21)

## Step 6: Generate Simplified Bidder Invoices

This step focuses on producing a clean, easy-to-read Excel workbook that contains individual bidder invoices.  
Each sheet in the workbook represents one bidder, with only the essential columns for payments and fines. A total row is added automatically for 
quick reference.


In [17]:
import os

# Define only the columns that actually exist in your dataset
invoice_columns = [
    'Order Number',
    'Date Bid',
    'Bidder',
    'Account',
    'Number of Pages',
    'Bidder_Pay_KES',
    'Fine_USD',
    'Paid_Status',
    'Duplicate_Flag'
]

# Create a smaller dataframe with just these columns
bidders_df = df[invoice_columns].copy()

# Create the output folder if missing
os.makedirs("output", exist_ok=True)

# Define output path for the final Excel workbook
output_path = "output/Bidder_Invoices_Final.xlsx"

# Write each bidder’s data into their own sheet
with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
    for bidder, bidder_data in bidders_df.groupby('Bidder'):
        if pd.notna(bidder):
            # Add a TOTAL row at the bottom
            total_row = pd.DataFrame({
                'Order Number': ['TOTAL'],
                'Date Bid': [''],
                'Bidder': [bidder],
                'Account': [''],
                'Number of Pages': [bidder_data['Number of Pages'].sum()],
                'Bidder_Pay_KES': [bidder_data['Bidder_Pay_KES'].sum()],
                'Fine_USD': [bidder_data['Fine_USD'].sum()],
                'Paid_Status': [''],
                'Duplicate_Flag': ['']
            })
            final_bidder = pd.concat([bidder_data, total_row], ignore_index=True)
            final_bidder.to_excel(writer, sheet_name=str(bidder)[:25], index=False)

print(f"✅ All bidder invoices saved successfully to {output_path}")


✅ All bidder invoices saved successfully to output/Bidder_Invoices_Final.xlsx
